In [1]:
import query_examples as qe
import utils
import pandas as pd
from rdflib import Graph, URIRef
from rdflib.parser import Parser



In [2]:
rdf_graph, label_uri_dict, uri_label_dict = utils.unpickle_resources()

In [3]:
PREFIXES = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX emmo: <http://emmo.info/emmo#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX echem: <http://emmo.info/electrochemistry#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dc: <http://purl.org/dc/elements/1.1/> 
PREFIX battery: <http://emmo.info/battery#> 
PREFIX schema:<https://schema.org/>
"""


## Search names by ORCID

In [5]:
orcid = "https://orcid.org/0000-0003-2954-1233"

query2 = PREFIXES  + f"""
SELECT ?givenName ?familyName ?email

WHERE {{
    <{orcid}> schema:familyName ?familyName;
            schema:givenName ?givenName;
            schema:email ?email.
}}
"""
qres = rdf_graph.query(query2)
list(qres)


[(rdflib.term.Literal('Eibar'),
  rdflib.term.Literal('Flores'),
  rdflib.term.Literal('eibar.flores.cedeno@sintef.no'))]

## Search expert by name

## Search all organizations

In [7]:
query4 = PREFIXES  + f"""
SELECT ?name ?id

WHERE {{
    ?id rdf:type schema:ResearchOrganization;
                schema:name ?name;
}}
"""
qres = rdf_graph.query(query4)


experts = {}
for result in qres:
    name_str = str(result[0])
    iri = str(result[1])
    experts[name_str] = iri

print(experts)

{'National Research Council': 'https://ror.org/04zaypm56', 'Paul Scherrer Institute': 'https://ror.org/03eh3y714', 'Karlsruhe Institute of Technology': 'https://ror.org/04t3en479', 'The Foundation for Industrial and Technical Research': 'https://ror.org/01f677e56', 'European Synchotron Radiation Facility': 'https://ror.org/02550n020', 'French Alternative Energies and Atomic Energy Commission': 'https://ror.org/00jjx8s55', 'Fraunhofer Society': 'https://ror.org/05hkkdn48', 'French national synchrotron facility': 'https://ror.org/01ydb3330', 'National Institute of Chemistry': 'https://ror.org/050mac570', 'Spanish National Research Council': 'https://ror.org/02gfc7t72', 'French National Centre for Scientific Research': 'https://ror.org/02feahw73', 'Institut Laue?Langevin': 'https://ror.org/01xtjs520', 'CIDETEC Foundation': 'https://ror.org/03vgz7r63', 'Research Centre Julich': 'https://ror.org/02nv7yv05'}


## Search all cells

In [5]:
query5 = PREFIXES  + f"""
SELECT ?instance ?creator
WHERE {{
    ?instance rdf:type/rdfs:subClassOf* <http://emmo.info/battery#battery_68ed592a_7924_45d0_a108_94d6275d57f0>;
                schema:creator ?creator
    }}
"""
qres = rdf_graph.query(query5)
print(list(qres))

[(rdflib.term.URIRef('http://rdf.batterymodel.com/96a8c962-cf63-40bf-a6c0-15f7b078e3e3'), rdflib.term.URIRef('https://orcid.org/0000-0002-8758-6109')), (rdflib.term.URIRef('http://rdf.batterymodel.com/b1bf0006-8006-4367-9625-ca869fed1684'), rdflib.term.URIRef('https://orcid.org/0000-0002-8758-6109')), (rdflib.term.URIRef('http://rdf.batterymodel.com/ed930bda-c2b6-4ade-b5cd-4d3102002edc'), rdflib.term.URIRef('https://orcid.org/0000-0003-2954-1233')), (rdflib.term.URIRef('http://rdf.batterymodel.com/ce40ac3a-6ef4-47ab-bfa4-d30e5c2fa39b'), rdflib.term.URIRef('https://orcid.org/0000-0002-8758-6109')), (rdflib.term.URIRef('http://rdf.batterymodel.com/6f1216b8-8b52-4340-ac12-f8effe4be910'), rdflib.term.URIRef('https://orcid.org/0000-0003-2954-1233'))]


In [11]:
query5 = PREFIXES  + f"""
SELECT ?instance ?creator
WHERE {{
    ?instance schema:creator ?creator
    }}
"""
qres = rdf_graph.query(query5)
print(list(qres))

[(rdflib.term.URIRef('http://rdf.batterymodel.com/96a8c962-cf63-40bf-a6c0-15f7b078e3e3'), rdflib.term.URIRef('https://orcid.org/0000-0002-8758-6109'))]


## Search all cells by expert orcid

In [53]:
orcid = URIRef("https://orcid.org/0000-0002-8758-6109")

query5 = PREFIXES  + f"""
SELECT ?instance ?givenName ?familyName ?email ?date
WHERE {{
    ?instance rdf:type battery:battery_68ed592a_7924_45d0_a108_94d6275d57f0;
        schema:creator <{orcid}>;
        schema:productionDate ?date.
    <{orcid}>   schema:familyName ?familyName;
                schema:givenName ?givenName;
                schema:email ?email.

    }}
"""
qres = rdf_graph.query(query5)
results_dict = {"CellID":[], "Researcher":[], "Email":[], "Date":[]}

for hit in qres:
        string_rep = [str(rdf_obj) for rdf_obj in hit]
        results_dict["CellID"].append(string_rep[0])
        results_dict["Researcher"].append(string_rep[1]+string_rep[2])
        results_dict["Email"].append(string_rep[3])
        results_dict["Date"].append(string_rep[4])

print(results_dict)



{'CellID': ['http://rdf.batterymodel.com/96a8c962-cf63-40bf-a6c0-15f7b078e3e3'], 'Researcher': ['SimonClark'], 'Email': ['simon.clark@sintef.no'], 'Date': ['2023-05-02']}


## Search all cells by organization ROR

In [54]:
ror = "https://ror.org/01f677e56"

query5 = PREFIXES  + f"""
SELECT ?instance ?name ?date ?personID
WHERE {{
    ?instance rdf:type battery:battery_68ed592a_7924_45d0_a108_94d6275d57f0;
        schema:manufacturer <{ror}>;
        schema:creator ?personID;
        schema:productionDate ?date.
    <{ror}>  schema:name ?name;

    }}
"""
qres = rdf_graph.query(query5)
results_dict = {"CellID":[], "Organization":[], "Date":[], "ResearcherID":[]}

for hit in qres:
        string_rep = [str(rdf_obj) for rdf_obj in hit]
        results_dict["CellID"].append(string_rep[0])
        results_dict["Organization"].append(string_rep[1])
        results_dict["Date"].append(string_rep[2])
        results_dict["ResearcherID"].append(string_rep[3])

print(results_dict)



{'CellID': ['http://rdf.batterymodel.com/96a8c962-cf63-40bf-a6c0-15f7b078e3e3'], 'Organization': ['The Foundation for Industrial and Technical Research'], 'Date': ['2023-05-02'], 'ResearcherID': ['https://orcid.org/0000-0002-8758-6109']}


In [55]:
utils.query_cell_by_organization(rdf_graph, ror="https://ror.org/01f677e56")

AttributeError: module 'utils' has no attribute 'query_cell_by_organization'